In [ ]:
!pip install nltk  

In [ ]:
import glob
import nltk
import pandas as pd
import math 
import json
import os 

if(os.path.isdir('bbc/') == False):
  !unzip bbc.zip
if(os.path.isdir('result/') == False):
  os.mkdir('result/')
  
nltk.download("stopwords")

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from collections import defaultdict

Archive:  bbc.zip
   creating: bbc/
  inflating: __MACOSX/._bbc          
  inflating: bbc/.DS_Store           
  inflating: __MACOSX/bbc/._.DS_Store  
  inflating: bbc/query.json          
  inflating: __MACOSX/bbc/._query.json  
   creating: bbc/entertainment/
  inflating: __MACOSX/bbc/._entertainment  
   creating: bbc/business/
  inflating: __MACOSX/bbc/._business  
   creating: bbc/sport/
  inflating: __MACOSX/bbc/._sport    
   creating: bbc/politics/
  inflating: __MACOSX/bbc/._politics  
  inflating: bbc/relevanceJudgment.json  
  inflating: __MACOSX/bbc/._relevanceJudgment.json  
  inflating: bbc/README.TXT          
  inflating: __MACOSX/bbc/._README.TXT  
   creating: bbc/tech/
  inflating: __MACOSX/bbc/._tech     
  inflating: bbc/entertainment/048.txt  
  inflating: __MACOSX/bbc/entertainment/._048.txt  
  inflating: bbc/entertainment/049.txt  
  inflating: __MACOSX/bbc/entertainment/._049.txt  
  inflating: bbc/entertainment/.DS_Store  
  inflating: __MACOSX/bbc/entertain

In [ ]:
stwords = set(stopwords.words('english'))
stemmer = PorterStemmer()
invertedIndexConst = defaultdict(set)
invertedIndex = defaultdict(list)
termDocumentMatrix = defaultdict(dict)
weightMatrix = defaultdict(dict)
sumVector = defaultdict(float)
filesList = glob.glob("bbc/*/*.txt")

unwantedChar = "!@#$;:!*%)(&^~.,/?[{}]+='"


In [ ]:
!echo "DocName" > fileList.txt; 
with open('fileList.txt', 'a') as filehandle:
    filehandle.writelines("%s\n" % fileName for fileName in filesList)
    

In [ ]:
def textPreProcessing(textData):
  textData = textData.replace('\n', ' ')
  textData = textData.replace('"','')
  textData = textData.lower()
  textData = textData.replace('-',' ')
  
  for char in unwantedChar:
    textData = textData.replace(char,"")

  textDataWithoutStopWords = [word for word in textData.split() if not word in stwords]
  textDataWithStemming = [stemmer.stem(word) for word in textDataWithoutStopWords]
  processedText = ' '.join(map(str, textDataWithStemming))

  return processedText

In [ ]:
for fileName in filesList:

  fileText = open(fileName).read()
  finalFileText = textPreProcessing(fileText)

  for word in finalFileText.split():
    invertedIndexConst[word].add(fileName)
    if( fileName in termDocumentMatrix[word]):
      termDocumentMatrix[word][fileName] = termDocumentMatrix[word][fileName] + 1
    else:
      termDocumentMatrix[word][fileName] = 1

for key in sorted(invertedIndexConst.keys()):
  l = list(invertedIndexConst[key])
  l.sort()
  invertedIndex[key] = l

In [ ]:
dataFrameForInvertedIndex = pd.DataFrame({key: pd.Series(value) for key, value in invertedIndex.items()})
dataFrameForInvertedIndex.to_csv('invertedIndex.csv', encoding='utf-8', index=False)
dataFrameForTermDocumentMatrix = pd.DataFrame(termDocumentMatrix).fillna(0)
dataFrameForTermDocumentMatrix.to_csv('termDocumentMatrix.csv')

# **TERM DOCUMENT MATRIX**


In [ ]:
dataFrameForTermDocumentMatrix

,green,set,sight,world,titl,mauric,aim,wipe,pain,lose,olymp,100m,athen,win,fourth,championship,crown,summer,settl,bronz,greec,behind,fellow,american,justin,gatlin,franci,obikwelu,portug,realli,hurt,look,medal,mistak,lost,thing,said,race,birmingham,friday,...,ballantin,snack,dunkin,donut,baskin,robbin,wsj,consolid,dealt,18bn,ft,19bn,woodley,delic,exorbit,nanj,auto,74,dayanidhi,maran,95,£106bn,communist,manmohan,singh,kobita,desai,fdi,ernst,sanjay,mehta,bharti,televentur,singapor,essar,hong,kong,hutchison,whampoa,sterl
bbc/sport/003.txt,9.0,2.0,1.0,6.0,4.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,3.0,4.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,6.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
bbc/sport/006.txt,2.0,0.0,0.0,3.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
bbc/sport/010.txt,1.0,0.0,0.0,7.0,1.0,1.0,1.0,0.0,0.0,0.0,5.0,4.0,2.0,2.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,1.0,0.0,2.0,1.0,0.0,0.0,0.0,1.0,1.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
bbc/tech/048.txt,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
bbc/entertainment/001.txt,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
bbc/entertainment/020.txt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
bbc/entertainment/038.txt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
bbc/entertainment/047.txt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
bbc/entertainment/030.txt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# **INVERTED INDEX**

In [ ]:
dataFrameForInvertedIndex

,000,002,01,019sec,02,022,028,03,0469,05,06,07,08,0800,1,10,100,1000,10000,100000,10000m,1000m,1004sec,100m,101,102bn,103m,104bn,107bn,108,1080,109bn,10bn,10k,10km,10sec,10th,11,11000,110000,...,£479m,£484bn,£4bn,£4m,£500000,£50m,£516m,£52m,£53,£532m,£5390,£53m,£546m,£550m,£55bn,£55m,£564bn,£57bn,£59m,£6,£60000,£600m,£611000,£630,£63m,£6m,£707m,£7455,£75m,£7m,£8000,£85,£86m,£885m,£8bn,£8m,£95m,£960m,£96bn,£9m
0,bbc/tech/047.txt,bbc/sport/003.txt,bbc/business/006.txt,bbc/sport/018.txt,bbc/business/006.txt,bbc/business/039.txt,bbc/tech/015.txt,bbc/business/044.txt,bbc/sport/032.txt,bbc/business/044.txt,bbc/business/023.txt,bbc/business/023.txt,bbc/business/023.txt,bbc/tech/015.txt,bbc/business/010.txt,bbc/business/014.txt,bbc/business/021.txt,bbc/tech/023.txt,bbc/entertainment/024.txt,bbc/business/049.txt,bbc/sport/018.txt,bbc/sport/014.txt,bbc/sport/028.txt,bbc/business/017.txt,bbc/politics/048.txt,bbc/business/033.txt,bbc/entertainment/018.txt,bbc/business/035.txt,bbc/business/005.txt,bbc/business/022.txt,bbc/tech/013.txt,bbc/business/001.txt,bbc/politics/024.txt,bbc/sport/019.txt,bbc/sport/007.txt,bbc/sport/018.txt,bbc/entertainment/045.txt,bbc/business/004.txt,bbc/entertainment/006.txt,bbc/business/027.txt,...,bbc/business/003.txt,bbc/business/024.txt,bbc/politics/022.txt,bbc/entertainment/049.txt,bbc/politics/007.txt,bbc/politics/023.txt,bbc/business/042.txt,bbc/tech/012.txt,bbc/tech/004.txt,bbc/business/035.txt,bbc/entertainment/024.txt,bbc/business/017.txt,bbc/business/035.txt,bbc/politics/006.txt,bbc/business/033.txt,bbc/entertainment/018.txt,bbc/business/024.txt,bbc/business/005.txt,bbc/business/004.txt,bbc/business/004.txt,bbc/entertainment/017.txt,bbc/business/001.txt,bbc/entertainment/045.txt,bbc/tech/022.txt,bbc/entertainment/049.txt,bbc/politics/029.txt,bbc/entertainment/046.txt,bbc/tech/045.txt,bbc/business/004.txt,bbc/entertainment/003.txt,bbc/business/049.txt,bbc/politics/049.txt,bbc/politics/029.txt,bbc/politics/030.txt,bbc/politics/021.txt,bbc/entertainment/012.txt,bbc/entertainment/049.txt,bbc/business/038.txt,bbc/business/048.txt,bbc/entertainment/005.txt
1,NaN,NaN,bbc/business/022.txt,NaN,NaN,NaN,NaN,bbc/business/046.txt,NaN,bbc/tech/032.txt,NaN,bbc/business/046.txt,NaN,NaN,bbc/business/018.txt,bbc/business/023.txt,bbc/business/047.txt,bbc/tech/031.txt,bbc/entertainment/027.txt,bbc/politics/002.txt,bbc/sport/025.txt,bbc/sport/032.txt,NaN,bbc/sport/003.txt,bbc/tech/009.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bbc/sport/038.txt,NaN,NaN,bbc/business/025.txt,NaN,bbc/business/049.txt,...,NaN,NaN,NaN,bbc/politics/044.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bbc/entertainment/018.txt
2,NaN,NaN,bbc/business/044.txt,NaN,NaN,NaN,NaN,bbc/tech/032.txt,NaN,NaN,NaN,NaN,NaN,NaN,bbc/business/046.txt,bbc/business/031.txt,bbc/entertainment/025.txt,NaN,bbc/tech/015.txt,bbc/politics/042.txt,bbc/sport/030.txt,NaN,NaN,bbc/sport/006.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bbc/business/050.txt,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bbc/entertainment/049.txt
3,NaN,NaN,bbc/business/046.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bbc/business/049.txt,bbc/business/037.txt,bbc/entertainment/042.txt,NaN,bbc/tech/035.txt,bbc/tech/002.txt,bbc/sport/032.txt,NaN,NaN,bbc/sport/010.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bbc/entertainment/014.txt,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bbc/entertainment/048.txt,bbc/business/039.txt,bbc/politics/003.txt,NaN,NaN,bbc/tech/011.txt,bbc/sport/034.txt,NaN,NaN,bbc/sport/016.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bbc/entertainment/016.t

In [ ]:
columnsInTermDocumentMatrix = len(dataFrameForTermDocumentMatrix.columns)
rowInTermDocumentMatrix = len(filesList)

In [ ]:
def tfIdfWeight(termFreq, docFreq):
  weightTF = 1 + math.log(termFreq , 10)
  weightIDF = math.log(rowInTermDocumentMatrix/docFreq , 10)
  weightTFIDF = weightTF * weightIDF
  return weightTFIDF

In [ ]:
for word,fileList in dataFrameForTermDocumentMatrix.items():               
  for fileName,termCount in fileList.items():
    termFreq = termCount
    docFreq = len(invertedIndex[word])      
    if( termFreq == 0):
      weightMatrix[word][fileName] = 0
      continue 
    weightMatrix[word][fileName] = tfIdfWeight(termFreq, docFreq)
    sumVector[fileName]= sumVector[fileName] + (weightMatrix[word][fileName] * weightMatrix[word][fileName])

dataFrameForWeightMatrixWithoutNormalization = pd.DataFrame(weightMatrix)

In [ ]:
for word,fileList in dataFrameForTermDocumentMatrix.items():               
  for fileName,termCount in fileList.items():
    weightMatrix[word][fileName] = weightMatrix[word][fileName] / math.sqrt(sumVector[fileName])
    
dataFrameForWeightMatrixWithNormalization = pd.DataFrame(weightMatrix)

# **NORMALIZATION WEIGHT MATRIX**

In [ ]:
dataFrameForTermDocumentMatrix = pd.DataFrame(dataFrameForWeightMatrixWithNormalization).fillna(0)
dataFrameForTermDocumentMatrix.to_csv('WeightMatrix.csv')
dataFrameForWeightMatrixWithNormalization

,green,set,sight,world,titl,mauric,aim,wipe,pain,lose,olymp,100m,athen,win,fourth,championship,crown,summer,settl,bronz,greec,behind,fellow,american,justin,gatlin,franci,obikwelu,portug,realli,hurt,look,medal,mistak,lost,thing,said,race,birmingham,friday,...,ballantin,snack,dunkin,donut,baskin,robbin,wsj,consolid,dealt,18bn,ft,19bn,woodley,delic,exorbit,nanj,auto,74,dayanidhi,maran,95,£106bn,communist,manmohan,singh,kobita,desai,fdi,ernst,sanjay,mehta,bharti,televentur,singapor,essar,hong,kong,hutchison,whampoa,sterl
bbc/sport/003.txt,0.143873,0.04201,0.100256,0.042631,0.096209,0.107231,0.058925,0.117062,0.100256,0.086689,0.061129,0.068211,0.058925,0.054799,0.069883,0.061236,0.108573,0.065176,0.078041,0.086689,0.086689,0.088744,0.080596,0.053960,0.117062,0.172914,0.144868,0.152301,0.117062,0.065176,0.090426,0.036273,0.065176,0.090426,0.093258,0.061236,0.004465,0.101027,0.078041,0.073919,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
bbc/sport/006.txt,0.137792,0.00000,0.000000,0.050946,0.000000,0.200699,0.000000,0.000000,0.000000,0.000000,0.087940,0.098127,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.115944,0.077626,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.004938,0.081734,0.146066,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
bbc/sport/010.txt,0.090009,0.00000,0.000000,0.054083,0.073421,0.131102,0.072042,0.000000,0.000000,0.000000,0.097596,0.133603,0.093729,0.066998,0.000000,0.097405,0.000000,0.079685,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.163304,0.143120,0.000000,0.079685,0.000000,0.057697,0.079685,0.000000,0.000000,0.000000,0.004196,0.069463,0.140937,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
bbc/tech/048.txt,0.076669,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.122518,0.000000,0.000000,0.000000,0.000000,0.000000,0.004650,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
bbc/entertainment/001.txt,0.075739,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
bbc/entertainment/020.txt,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [ ]:
with open('bbc/query.json') as queryFile:
  queryList = json.load(queryFile) 

with open('bbc/relevanceJudgment.json') as relevanceJudgmentFile:
  relevanceJudgment = json.load(relevanceJudgmentFile) 

In [ ]:
def writeToJsonFile(jsonObject, filename):
  filepath = 'result/'+filename+'.json'    
  jsonObject = json.dumps(jsonObject, indent = 4) 
 
  with open(filepath, "w") as outfile: 
      outfile.write(jsonObject)

In [ ]:
def queryProcessingFunction(query):
  queryTermFreq = defaultdict(int)
  queryWeightVector = defaultdict(float)
  querySumVector = 0

  for word in query.split():
    queryTermFreq[word] = queryTermFreq[word] + 1;
  
  for word, termFreq in queryTermFreq.items():
    if(word not in invertedIndex):
      queryWeightVector[word] = 0
      continue
    docFreq = len(invertedIndex[word])
    queryWeightVector[word] = tfIdfWeight(termFreq, docFreq)
    querySumVector = querySumVector + (queryWeightVector[word]*queryWeightVector[word])
 
  if(querySumVector != 0):
    for word, termFreq in queryTermFreq.items():
      queryWeightVector[word] = queryWeightVector[word] / math.sqrt(querySumVector)

  return queryWeightVector

In [ ]:
def precisionCalculation(queryId, cosineSimilarityRanking):
  relevantCount = 0
  retrieveCount = 0
  precisionSum = 0 
  for fileName, consineValue in cosineSimilarityRanking:
    retrieveCount = retrieveCount + 1
    if(fileName in relevanceJudgment[queryId]['related_document']):
      relevantCount = relevantCount + 1
      precisionSum = precisionSum + (relevantCount / retrieveCount)

  precision = precisionSum / len(relevanceJudgment[queryId]['related_document'])
  return precision

# **Query Result and Avg Precision**

In [ ]:
precisionSum = 0;
for queryId,query in queryList.items():
  processedQuery = textPreProcessing(query)
  queryWeightVector = queryProcessingFunction(processedQuery)

  cosineSimilarity = defaultdict(float)
  
  for fileName in filesList:
    cosineSimilarity[fileName] = 0

  for fileName in filesList:
    for word in processedQuery.split():
      if word in weightMatrix:
        cosineSimilarity[fileName] = cosineSimilarity[fileName] + (weightMatrix[word][fileName] * queryWeightVector[word])
    

  cosineSimilarityRanking = [[k,v] for k,v in cosineSimilarity.items()]
  cosineSimilarityRanking.sort(key=lambda x:x[1],reverse=True)
  sliceList = cosineSimilarityRanking[:10]

  precision = precisionCalculation(queryId, cosineSimilarityRanking)
  precisionSum = precisionSum + precision

  jsonObject = {
            '1. queryId': queryId,
            '2. query':query,
            '3. precision':precision, 
            '4. DocList':sliceList  
    }

  writeToJsonFile(jsonObject,queryId) 
  s1 = json.dumps(jsonObject)
  json_dict = json.loads(s1)
  print(json.dumps(json_dict, indent = 4, sort_keys=True))

{
    "1. queryId": "q0",
    "2. query": "how does us market affect economy growth and job market this year",
    "3. precision": 0.7038944695451097,
    "4. DocList": [
        [
            "bbc/business/022.txt",
            0.19242881249541358
        ],
        [
            "bbc/business/007.txt",
            0.17813824285938312
        ],
        [
            "bbc/business/006.txt",
            0.1523374052553602
        ],
        [
            "bbc/business/031.txt",
            0.13182743051888013
        ],
        [
            "bbc/business/050.txt",
            0.11460460578350878
        ],
        [
            "bbc/business/044.txt",
            0.1123436212672269
        ],
        [
            "bbc/business/016.txt",
            0.11094903654718509
        ],
        [
            "bbc/business/030.txt",
            0.10297105194168311
        ],
        [
            "bbc/tech/040.txt",
            0.09868684286060461
        ],
        [
            "bbc/busines

# **Mean Avg Precision**

In [ ]:
meanAvgPrecision = precisionSum / len(queryList)
print("Mean Average Precision : " + str(meanAvgPrecision))

Mean Average Precision : 0.7899625331397928
